### Collisional-radiative model for Argon Plasma
#### Kinetics fitting
See: https://doi.org/10.1063/1.3585688

In [1]:
import os
import copy
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from silx.io.dictdump import dicttoh5, h5todict

Inputs

In [2]:
path_to_rates = "./../kinetics/rates_raw.hdf5"

In [3]:
# Save path
out_path = "./../kinetics/"
os.makedirs(out_path, exist_ok=True)

Fit functions and parameters

In [4]:
ARR_PARAM = ("A", "beta", "Ta")

# Arrhenius law - Linear
def fun(T, *args):
  A, beta, Ta = args
  return A * np.exp(beta*np.log(T) - Ta/T)

# Arrhenius law - Log
def fun_ln(T, *args):
  logA, beta, Ta = args
  return logA + beta*np.log(T) - Ta/T

# Arrhenius law Jacobian - Log
def jac_ln(T, *args):
  return np.vstack([np.ones_like(T), np.log(T), -1/T]).T

# Curve fit arguments
cfit_args = {
  "f": fun_ln,
  "p0": [-20, 0, 1e3],
  "method": "trf",
  "jac": jac_ln,
  "bounds": (
    [-np.inf, -np.inf,   0], # Lower bounds
    [     -5,  np.inf, 1e6]  # Upper bounds
  )
}

# Fitting function
def fit_rate(T, k):
  indices = k > 0
  T = T[indices]
  k = np.log(k[indices])
  if (len(T) < 4):
    param = np.zeros((3,))
  else:
    param = curve_fit(
      xdata=T,
      ydata=k,
      **cfit_args
    )[0]
    param[0] = np.exp(param[0])
  return param

In [5]:
rates_raw = h5todict(path_to_rates)
T = rates_raw["T"]

In [6]:
rates_fit = copy.deepcopy(rates_raw)
for (name, rates) in rates_raw.items():
  if (name not in ("T", "A", "Rp", "ken")):
    shape = rates.shape[:-1]
    rates = rates.reshape(-1, len(T))
    param = np.stack([fit_rate(T, k) for k in rates], 1)
    param = {p: param[i].reshape(shape) for i, p in enumerate(ARR_PARAM)}
    rates_fit[name] = param

Compare fit rates

In [7]:
FIT_RATES = ("C", "K", "R", "S", "V")

In [8]:
def pick_rate():
  name = np.random.choice(FIT_RATES)
  size = rates_fit[name]["A"].size
  rate = 0.0
  while rate == 0.0:
    i = np.random.choice(size)
    rate = rates_fit[name]["A"].reshape(-1)[i]
  return name, i

def compute_rates(name, i):
  param = [rates_fit[name][p].reshape(-1)[i] for p in ARR_PARAM]
  k_fit = fun(T, *param)
  k_raw = rates_raw[name].reshape(-1, len(T))[i]
  return k_fit, k_raw

In [ ]:
for i in range(20):
  name, i = pick_rate()
  k_fit, k_raw = compute_rates(name, i)
  plt.plot(T, k_fit, label="Fit")
  plt.plot(T, k_raw, label="Raw")
  plt.title(f"{name}, i = {i} - Linear")
  plt.legend()
  plt.show()
  plt.close()
  plt.semilogy(T, k_fit, label="Fit")
  plt.semilogy(T, k_raw, label="Raw")
  plt.title(f"{name}, i = {i} - Log")
  plt.legend()
  plt.show()
  plt.close()

Save fit rates

In [10]:
dicttoh5(rates_fit, out_path + "/rates_fit.hdf5")